# Model deployment on AWS Sagemaker Endpoint

- [Hugging Face model](https://huggingface.co/arnaucas/wildfire-classifier)

In [5]:
import sagemaker
import boto3
import requests
from botocore.exceptions import NoCredentialsError
from sagemaker.huggingface import HuggingFaceModel

In [11]:
client = boto3.client('sagemaker')
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_session.region_name

BUCKET = "mlschool-arnaucasbas"

## Create endpoint

*Code mainly extracted from the "Deploy" tab on the Hugging Face model page.*

<div class="alert" style="background-color:#d0b700;"><strong>Note:</strong> 
    Uncomment the %%script cell magic line to execute this cell.
</div>

In [12]:
%%script false --no-raise-error

# Specify model name and task
hub = {
	'HF_MODEL_ID':'arnaucas/wildfire-classifier',
	'HF_TASK':'image-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

----!

## Inference

In [13]:
from sagemaker.serializers import DataSerializer
from PIL import Image

predictor.serializer = DataSerializer(content_type='image/x-image') # change to audio/x-audio for audio

img = './fire.jpg'

predictor.predict(img)

[{'score': 0.9975895881652832, 'label': 'fire'},
 {'score': 0.002410453511402011, 'label': 'nofire'}]

<div class="alert" style="background-color:#f52c1d;"><strong>Note:</strong> 
    Always remember to delete the endpoint at the end
</div>

In [16]:
predictor.delete_model()
predictor.delete_endpoint()

TODOs:
- [X] Endpoint creation
- [X] Code for programatically inference
- [ ] Build REST API ([CLUE](https://archive.md/zS7WK))
- [ ] Build custom inferencing script [(HERE)](https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb)

Extra resources:
- [1](https://github.com/huggingface/notebooks/tree/main/sagemaker)